## Final Project Submission

Please fill out:
* Student name: Pedro Jofre Lora
* Student pace: self paced
* Scheduled project review date/time: 2/14/2019 10:00AM EST
* Instructor name: Eli Thomas
* Blog post URL:


# Modeling King County Housing Prices Using MLR Supported with Cross-Validated RFE.
### Table of Contents
1. [Introduction](#1) <br>
2. [Importing and Cleaning Data](#2) <br>
3. [Exploring and Modifying Data](#3) <br>
4. [Modeling Data](#4) <br>
5. [Interpreting Data](#5)

<a id="1"></a>
## 1. Introduction
This is the introduction where you explain the relevant information given the question that was asked. 

<a id="2"></a>
## 2. Importing and cleaning data (Obtain and Scrub)
### 2.1 Importing Data

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

# Import the data
kc = pd.read_csv('kc_house_data.csv', index_col = 'id')
kc.columns = [col_name.title() for col_name in list(kc.columns)]
kc.info()

In [ ]:
pd.DataFrame(
    data = kc.nunique().sort_values(), 
    columns = ['Counts']
    ).transpose()

In [ ]:
kc.describe()

### 2.2 Define problems
1.  Date, View, and Sqft_basement are incorrect types.
2.  Waterfront and zipcode likely need to be converted to categorical.
3.  Condition and Grade might need to be converted to categorical, unless the numbers are a monotonic scale (scale of 1-5 with 5 being the "best").
4.  Investigate Yr_Renovated.
5.  Deciding what to do with the date.
6.  Deciding what to do with the lat and long values.

Data could be dropped from View, and it should also be converted to a binary category since the description is "has been viewed", and not "the number of times the propery has been viewed".

Waterfront and zipcode may not need to be converted to categorical and could be tossed out if the distributions of the groups is equal. If the distributions are not equal then they'll both be changed to categorical. It might be best to run separate analyses for the categories if there's enough data, since the relationships may change in shape given the value of a category.

Condition and Grade likely will not need to be converted to categorical, though we must assume that they are monotonic scales.

There is a surprisingly low mean and a very high std for Yr_Renovated, which may indicate that the data may not accurately represent what's going on. It may be the case that very few homes have renovations on file. If this is the case, then Yr_renovated should be changed to Renovated, which is a binary category indicating whether or not a renovation is on file.

The date will likely have a large impact on predicting the final sale price. I'll investigate to see if there is an impact, and then decide what to do.

Changing Lat and Long data into rectangular sectors can approximate neighborhoods that can then be used as categories. It would be best to look at a map of king county to see if there are distinct neighborhoods that could be drawn instead. This would be an interesting function...


### 2.3 Resolving problems
#### 2.3.1. Change Date and Sqft_Basement to correct types. Drop Data as necessary. Make a separate Basement as a category.

In [ ]:
kc.Sqft_Basement.replace(to_replace = '?', 
                         value = np.nan, 
                         inplace = True)
kc.Sqft_Basement = kc.Sqft_Basement.apply(pd.to_numeric)
kc = kc[kc.Sqft_Basement.notnull()]
kc = kc[kc.View.notnull()]
kc.View = kc.View.where(kc.View <= 0, other = 1)
kc['View'] = (kc.View > 0).astype('category')

In [ ]:
kc['Basement'] = (kc.Sqft_Basement>0).astype('category')

In [ ]:
kc.Date = pd.to_datetime(kc.Date)

#### 2.3.2 Change Waterfront and zipcode to categories if appropriate

In [ ]:
waterfront_grouping = kc.groupby(by = 'Waterfront')
waterfront_grouping.groups.keys()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_style(style = 'dark')
ax1 = waterfront_grouping.get_group(0).Price.plot.hist(density = True)
waterfront_grouping.get_group(1).Price.plot.hist(density = True,
                                            secondary_y = True,
                                            alpha = 0.3,
                                            ax = ax1)
ax1.set_xlabel('Price')
plt.show()

The distributions are clearly different in shape, so waterfront will be converted to a category.

In [ ]:
kc.Waterfront.fillna(0.0,inplace = True) # Assume that a null listing means the house is not waterfront
kc.Waterfront = (kc.Waterfront>0).astype('category')

In [ ]:
kc.groupby('Zipcode').Price.median().plot.box(figsize = (3,5));

The medians of the groups are too spread out to believe that zipcode might not have a significant impact on the final price of the house. It will be converted to a category.

In [ ]:
kc.Zipcode = kc.Zipcode.astype('category')

#### 2.3.3 Changing Grade and Condition to categories.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (12,4))
kc.groupby('Grade').Price.median().plot(ax = ax1, title = 'Price by Grade');
kc.groupby('Condition').Price.median().plot(ax = ax2, title = 'Price by Condition')
ax1.set_ylabel('Price')
ax2.set_ylabel('Price')
plt.show()

Price as a function of Condition is not easily approximated by a simple function, though this is odd. I'll investigate price by condition more closely to see if there's a reason behind the deviation.

In [ ]:
kc.groupby('Condition').Price.agg(['count','max','min','mean','median'])

The min and max follow a monotonically increasing trend. The scale is most likely meant to be monotonic, so I'll treat it as such and leave Condition as a numerical feature. 

It is also strange that the lowest value for Grade has a higher price than the two following values for grade. I'll also investigate that further.

In [ ]:
kc.groupby('Grade').Price.agg(['count','max','min','mean','median'])

There is only one record for a house with a grade of 3. This record will be eliminated since we don't know if this house is an outlier.

In [ ]:
kc.drop(kc.loc[kc.Grade == 3].index, inplace = True)

#### 2.3.4  Changing Yr_Renovated to Yrs_Since_Renovation.
This is relatively straightforward. The complicated part is that many of the homes have a value of 0.0 for the year renovated, which is nonsensical. It would be good to investigate how many homes have a renovation date on record.

In [ ]:
num_unknown_rennovations = kc.Yr_Renovated.replace(to_replace = 0,
                                                   value = np.nan).isna().sum()
num_renovated = (len(kc) - num_unknown_rennovations)
'The number of houses with documented renovations is {0:G}'.format(num_renovated)

Since there are so few homes and so many features, I don't think it makes sense to treat this as a continuous variable for those homes that have been renovated.

In [ ]:
kc.Yr_Renovated.fillna(0, inplace = True)
kc['Renovated'] = kc.Yr_Renovated.where(kc.Yr_Renovated <= 0, other = 1)
kc['Renovated'] = (kc.Renovated > 0).astype('category')
kc.drop(labels = 'Yr_Renovated', axis = 1, inplace = True)

#### 2.3.5 Dealing with the date.

My inclination here is that prices are higher in the summer months because demand is higher. I'll look at a 30 day rolling window to spot trends in the median price, and also resample the data by month to also look for trends.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12,4))
kc.groupby('Date').Price.median().rolling(30).median().plot(ax = ax1, title = 'Rolling Window')
kc.reset_index().set_index(['Date']).resample('m').Price.median().plot(ax = ax2, title = 'Resampled Data')
plt.show()

# Calculate the maximum 'swing' in price as compared to the median
price_adjustment = (kc.reset_index().set_index(['Date']).resample('m').Price.median() / kc.Price.median() - 1)*100
max_swing = price_adjustment.max()-price_adjustment.min()
'The maximum swing in adjusted price is {0:2.2f}%'.format(max_swing)

The maximum swing on the median price is a hefty 11.61%! The model will have to account for the time of the year in order to be more accurate. The easiest way to do this will be to resample by month and then turn into a category. Complexity can be added to the model later by fitting a function to the date, instead of simply categorizing by month.

In [ ]:
kc['Month'] = kc.Date.dt.month_name().astype('category')
kc.drop('Date', axis = 1, inplace = True)

#### 2.3.6 Deciding what to do with lat and long values

Here's a map of King County with zip-code boundaries:
<img src="https://www.kingcounty.gov/~/media/operations/GIS/maps/vmc/images/zipcodes_westKC_586.ashx?la=en" alt="An image of King County with zip-code boundaries" title="King Country Zip Codes" />
<br><br><br>
And, for comparison's sake, here's a map of Manhattan with zip-code boundaries:
<img src="https://www.propertyshark.com/Real-Estate-Reports/wp-content/uploads/2012/10/infographic-zip-codes-3.png" alt="An image of Manhattan with zip-code boundaries" title="Manhattan Zip Codes" />

Zip code boundaries trace a mixture of natural boundaries and, I suspect, socioeconomic boundaries. This is corroborated by the Manhattan zipcodes, where neighborhoods are roughly approximated by zip codes (I live in NYC, so I'm more familiar with the neighborhoods here). Though zip codes are not a perfect stand-in for neighborhoods, they are adequate for a first pass at this model. If I have time, I will attend to making discrete quadrants that better capture pricing at the neighborhood level. 


In [ ]:
kc.drop(labels = ['Lat', 'Long'], axis = 1, inplace = True)

To finish up importing the data, I'll check the metadata again:

In [ ]:
kc.info()

That looks significantly better now. Some series' types could be changed to improve efficiency, but I'm not worried about it at this point since the dataframe is small.

<a id="3"></a>
## 3. Exploring and Modifying Data

First I'll look at violin plots of Price for the predictors that have categorical values. Then I'll look at distributions for continuous predictors. Before I go much further, I'll look for correlations between the continuous predictors and make some decisions about what data to drop. Finally, I'll look at scatter plots of Price vs continous predictors. In order to make this less repetitive, I'll use ipywidgets to create interactive displays. I'll dive deeper as needed along the way.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### 3.1  Violin plots

In [ ]:
feature_list = list(kc.select_dtypes('category').columns)
feature_list.remove('Zipcode')
hue_list = feature_list.copy()
hue_list.append('None')

In [ ]:
@interact
def violin_plot(x=feature_list, 
                yscale = ['log', 'linear'],
                hue = hue_list):
    
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    if (hue != 'None') & (hue != x):
        title = "Price vs. " + x + " sorted by " + hue
        hue = kc[hue]
    else:
        hue = None
        title = "Price vs. " + x
    
    
    sns.violinplot(x = kc[x], 
                   y = kc.Price, 
                   cut = 0,
                   scale = 'area',
                   inner = 'box',
                   hue = hue,
                   ax = ax,
                   )
    plt.yscale(yscale)
    plt.title(title)


### 3.2  Histograms of numerical features

After making the histograms I noticed that there's a house with 33 bedrooms whose price and living area do not corroborate the number of bedrooms. I'll drop it from the dataframe since I can't be certain that it's supposed to have 3 bedrooms instead. That being said, I want to investigate the descriptive values again to make sure that I haven't missed any other obvious outliers. 

In [ ]:
kc.describe()

Everything else looks okay. I'll get rid of the house with 33 bedrooms and then check again.

In [ ]:
kc.drop(labels = kc.loc[kc.Bedrooms == 33].index, inplace = True)
kc.Bedrooms.describe()

In [ ]:
@interact
def hist_plot(data = list(kc.select_dtypes(include = [np.number]).columns),
             log_data = [False, True]):
    
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    title = "Distribution of " + data
    
    if log_data:
        data = kc[data].replace(0,0.1)
        data = data.apply(np.log10)
    else:
        data = kc[data]
    sns.distplot(data,
                 ax = ax)
    plt.title(title)
    plt.show()

### 3.3  Correlation matrix

In [ ]:
predictors = kc.drop(labels = 'Price', axis = 1)
sns.set_style('white')
corr = predictors.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.5, vmin = -0.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot = True)
plt.show()
sns.set_style()

The correlation matrix shows that many of these variables correlate with each other. I suppose I'm not surprised by this. After all, a more expensive house tends to have more bedrooms, bathrooms, space, and floors. Additionally, it is likely to have a higher grade and be newer. So, those things tend to go together and intuitively they will be correlated.

From the description of the columns, I don't think that we need to keep all three of: Sqft_living, Sqft_Above, and Sqft_Basement. Sqft_Above indicates the area of the house above the basement. It is likely that the sum of the area above the basement and the basement area is almost always the "living area". I'll check this assumption quickly. If this mostly holds true, then I'll drop "living area", if not then I'll drop "above".

In [ ]:
print(((kc.Sqft_Above+kc.Sqft_Basement)==kc.Sqft_Living).sum()) #This is the number of houses whose area above and below matches total area
print(len(kc.Price))

How wonderful! It's a perfect match, so I'll drop Sqft_Living since it's accounted for in the sum of basement and above area.

In [ ]:
kc.drop('Sqft_Living', axis = 1, inplace = True)

In [ ]:
predictors.drop('Sqft_Living', axis = 1, inplace = True)

Now I'll look at predictors that measure the difference between the house itself and the houses of neighbors. The interesting thing to look at here, I think, is the houses that are NOT like their neighbors. I imagine that this is where differences arise, so it might be best to create new categories for these comparators that determines if the neighbors are bigger, smaller, or the same. To do this, I'll look at the differences between the house itself and its neighbors for each of these categories.

#### 3.3.1 Analyzing data of neighbors

In [ ]:
neighbors_data = pd.DataFrame()
neighbors_data['Lot_Difference'] = (predictors.Sqft_Lot - predictors.Sqft_Lot15)/predictors.Sqft_Lot15
neighbors_data['Living_Difference'] = (predictors.Sqft_Above + predictors.Sqft_Basement - predictors.Sqft_Living15)/predictors.Sqft_Living15

In [ ]:
neighbors_data.describe()

The descriptive data alone tells us that there are outliers in the data. I'll look at histograms to better understand the data. By looking at layered slices of the data I may be able to determine where the null hypothesis $\mu_{low}$ = $\mu_{mid}$ = $\mu_{high}$ can be rejected.

In [ ]:
data = neighbors_data['Living_Difference']
data.loc[(data <= data.quantile(0.0))].index

In [ ]:
@interact
def neighbor_difference(
              data = ['Lot_Difference', 'Living_Difference'],
              lower_cutoff = (0,0.5,0.05),
              upper_cutoff = (0.5,1,0.05)):
    
    fig, (ax1) = plt.subplots(1,1,figsize = (16,9))
    title = "Distribution of " + data
    
    data = neighbors_data[data]
    data_low = kc.loc[data.loc[(data <= data.quantile(lower_cutoff))].index].Price.dropna()
    data_mid = kc.loc[data.loc[(data > data.quantile(lower_cutoff)) & (data < data.quantile(upper_cutoff))].index].Price.dropna()
    data_high = kc.loc[data.loc[(data >= data.quantile(upper_cutoff))].index].Price.dropna()
    sns.distplot(data_low.apply(np.log10), kde = True, hist_kws=dict(alpha=0.3), ax = ax1)
    sns.distplot(data_mid.apply(np.log10), kde = True, hist_kws=dict(alpha=0.3), ax = ax1)
    sns.distplot(data_high.apply(np.log10), kde = True, hist_kws=dict(alpha=0.3), ax = ax1)
    n_text = "n Low:{0:d}\nn Mid:{1:d}\nn High:{2:d}".format(len(data_low),len(data_mid),len(data_high))
    ax1.text(-0.05, 
             0.9, 
             n_text, 
             horizontalalignment='left',
             verticalalignment='top',
             fontsize = 16,
             transform=ax.transAxes)
    ax1.set_title(title)
    plt.legend(['Low','Middle','High'], prop = {'size':14})
    plt.show()

It does not appear that a difference in lot size is a good predictor of price even at the extremes of the distributions. It makes sense not to include lot size differences in the final model.
A difference in living area compared to the neighbors does, however, have a pretty significant effect on the price of a house. This is easily visualized by splitting the data into the bottom and top 5%. Given this finding, I'll make categories for houses in the top and bottom 5% of living area difference.

Of course, if this turns out not to be a significant factor in the model then my visual analysis is incorrect. I'll assess this after the model is built.

In [ ]:
data = neighbors_data['Living_Difference']
kc['Living_Top'] = data >= data.quantile(0.95)
kc.Living_Top = kc.Living_Top.astype('category')
kc['Living_Bottom'] = data <= data.quantile(0.05)
kc.Living_Bottom = kc.Living_Bottom.astype('category')
kc.drop(labels = ['Sqft_Living15', 'Sqft_Lot15'], axis = 1, inplace = True)

### 3.4 Dropping Data
Now I have to make some decisions about which numerical series to drop. I'll look at the correlations first.

In [ ]:
kc.corr().Price.sort_values(ascending = False)

Condition, Yr_Built, and Sqft_Lot are not well correlated, so they will be dropped from the dataframe. Floors are also not well correlated and it is more highly correlated with Sqft_above, so that will be dropped too.

In [ ]:
kc.drop(labels = ['Condition','Yr_Built', 'Sqft_Lot','Floors'], axis = 1, inplace = True)

In [ ]:
sns.set_style('white')
corr = kc.drop(labels = 'Price', axis = 1).corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.5, vmin = -0.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot = True)
plt.show()
sns.set_style()

This is better, but there is still correlation between predictors. I will drop more features from the dataframe if this multicollinearity becomes a problem in the model. For now, I'm content with the results and I'll move on to analyzing the effect of the zip code and month on price.
<br>
<br>
### 3.5  Managing Month and Zipcode
Zipcode is a categorical series with too many unique values to be functionally useful. RFE will take forever if there are too many dummy variables, and deciding which dummy variables to use is somewhat arbitrary and based on the total number of features. It might be best to perform target encoding with the zipcode. I'll do the same for months, though I could leave months as a categorical variable and hot-encode if the correlation is low.

In [ ]:
zip_means = kc.groupby('Zipcode')['Price'].mean()
month_means = kc.groupby('Month')['Price'].mean()
kc['Zipcode_Means'] = kc.Zipcode.map(zip_means)
kc['Month_Means'] = kc.Month.map(month_means)
print('Zipcode Means : Price\t' + str(kc.corr().loc['Zipcode_Means','Price']))
print('Month : Price\t\t' + str(kc.corr().loc['Month_Means','Price']))

The correlation between Zipcode Means and Price is high, so I'll keep the target encoding and drop the Zipcodes. The correlation between Month and Price is very low, so I'll one-hot encode.

In [ ]:
kc.drop('Zipcode', axis = 1, inplace = True)

In setting the dummies I found out that the dataset has duplicate entries for single ids. I'll drop the duplicates and reset the index.

In [ ]:
kc = kc.reset_index().drop_duplicates(subset = 'id')
kc['id'] = kc.id.astype('int')
kc = kc.set_index('id')

In [ ]:
month_dummies = pd.get_dummies(kc.Month)
kc = kc.join(month_dummies,on = 'id')
kc.drop(labels = ['Month', 'Month_Means'], axis = 1, inplace = True)

I will also drop the last month, September, since it is encoded in the other month variables (if not every other month, then it must be September). This will ensure that statsmodel doesn't inject another intercept unknowingly. 

In [ ]:
kc.drop(labels = 'September', axis = 1, inplace = True)

#### 3.6 Converting categorical data to uint8
This is a silly step, but I have discovered that statsmodel behaves strangely when categorical data is passed to it. Statsmodel calculates an intercept when categorical data is passed into statsmodel appropriately even if the intercept has been explicitly removed.

In [ ]:
columns = kc.select_dtypes('category')
for column in columns:
    kc[column] = kc[column].astype('uint8')

In [ ]:
kc.info()

### 3.6 Scatter Plots
The data has been cleaned up and I now have a better sense of what to do when building the model. Now I can look at scatter plots to better understand the relatioship between the predictors and the target.

One thing that I'll explore are fits for discrete data based on the median at each value. This will give me a better sense of the function that best fits each curve, which I can leverage to linearize the data before I run MLS.

In [ ]:
@interact
def discrete_plot(x = ['Bedrooms', 'Bathrooms', 'Grade', 'Zipcode_Means'],
                  y_function = ['linear', 'log'],
                  x_function = ['linear', 'log', 'squared', 'square-root', 'exp']):
    
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    df = pd.DataFrame(kc.groupby(x).Price.median()).reset_index()
    x = df[x]
    y = df.Price
    def function_mapping(data, function):
        function_map = {'linear':data,
                            'log':data.apply(np.log10), 
                            'squared':data.apply(np.square),
                            'square-root':data.apply(np.sqrt), 
                            'exp':data.apply(np.exp)}
        return function_map[function]
    
    x = function_mapping(x,x_function)
    y = function_mapping(y,y_function)
    corr = np.corrcoef(x,y)
    corr_text = 'corr: {:1.3f}'.format(corr[1,0])
    sns.scatterplot(x = x, y = y)
    plt.text(.025, 
             0.95, 
             corr_text, 
             horizontalalignment='left',
             verticalalignment='top',
             fontsize = 16,
             transform=ax.transAxes)
    plt.show()
    

It appears that taking the log of the price data helps linearize overall. This also helps because the distribution of the price data is normal when it is log transformed. The following relationships have the best correlation values:

| Relationship                 | Correlation Value |
| ---------------------------- | :---------------: |
| log(Price) ~ log(Bedrooms)   | 0.871             |
| log(Price) ~ sqrt(Bathrooms) | 0.867             |
| log(Price) ~ $Grade^2$       | 0.998             |
| log(Price) ~ log(Zipcode)    | 0.992             |

In [ ]:
@interact
def scatter_plot(x = ['Sqft_Above','Sqft_Basement'],
                 x_function = ['linear', 'log', 'squared', 'square-root', 'exp'],
                 hue = ['None', 'Waterfront', 'View', 'Basement', 'Renovated']):
    
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    def function_mapping(data, function):
        function_map = {'linear':data,
                            'log':data.apply(np.log10), 
                            'squared':data.apply(np.square),
                            'square-root':data.apply(np.sqrt), 
                            'exp':data.apply(np.exp)}
        return function_map[function]
    x_data = kc.loc[kc[x] > 0][x]
    x_data = function_mapping(x_data,x_function)
    y_data = kc.loc[kc[x] > 0]['Price']
    y_data = y_data.apply(np.log10)
        
    if (hue != 'None'):
        hue = kc[hue]
    else:
        hue = None
    
    corr = np.corrcoef(x_data,y_data)
    corr_text = 'corr: {:1.3f}'.format(corr[1,0])
    plt.text(.025, 
             0.95, 
             corr_text, 
             horizontalalignment='left',
             verticalalignment='top',
             fontsize = 16,
             transform=ax.transAxes)
    
    sns.scatterplot(x = x_data, y = y_data, hue = hue, ax=ax)
    plt.show()

While not transforming the predictors yields the best correlation values, I'm concerned that there will be a non-random pattern in the residuals if they're left alone. Log transforming Sqft_Above and taking the square root of Sqft_Basement is more likely to yield a random residual pattern in both. This relationship and the correlation values is shown below.

|Relationship|Correlation Value|
|---|---|
|log(Price) ~ log(Sqft Above)| 0.586 |
|log(Price) ~ $SqftBasement^{1/2}$ | 0.374 |

<a id="4"></a>
## 4. Modeling Data

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

First, I'll transform the data based on the analysis that I performed above.

In [ ]:
kc_transformed = kc.copy()
transform = {'Price':np.log10, 
             'Bedrooms':np.log10, 
             'Bathrooms':np.sqrt, 
             'Grade':np.square, 
             'Sqft_Above':np.log10, 
             'Sqft_Basement':np.sqrt,
             'Zipcode_Means':np.log10}

for column in list(kc.columns):
    if column in transform.keys():
        kc_transformed[column] = kc_transformed[column].apply(transform[column])
kc_transformed.describe()

I will normalize the data since the range for each column is significantly different from the others.

In [ ]:
kc_transformed_scaled = kc_transformed.copy()
to_scale = kc_transformed_scaled.select_dtypes(include = [np.number]).columns
for column in list(to_scale):
    data = kc_transformed_scaled[column]
    kc_transformed_scaled[column] = (data - data.min())/(data.max()-data.min())
kc_transformed_scaled.agg(['min', 'max'])

Wonderful! Now I can build the model using the transformed and scaled data. I'll make training and testing sets to work with from now on. First I'll make the formula.

In [ ]:
predictors = kc_transformed_scaled.drop(labels = 'Price', axis = 1).copy()
target = kc_transformed_scaled.Price.copy()
predictor_list = list(predictors.columns)
formula = 'Price ~ ' + ' + '.join(list(predictors.columns)) + ' - 1'
print(formula)
print(len(predictors.columns))

This formula accounts for all 26 predictors. I'm concerned about two things: 1) this is way too many predictors, so the model may not be sensical, and 2) there is likely still correlation between predictors. It will be easy to see if there are indeed to many predictors - the model will have some predictors whose coefficients are not statistically significant if too many predictors are confounding the results. It should also be easy to see if correlation between predictors is causing a problem by checking the values of the coefficients against our understanding of the problem. For example, we know that more bedrooms, more bathrooms, and more space indicate a higher price. If any of these coefficients are negative, then it stands to reason that interactions between the predictors is causing a problem in the model. I'll perform a multiple linear regression and look at the outputs.

The train/test set is made below.

In [ ]:
from sklearn.model_selection import train_test_split

predictor_train, predictor_test, target_train, target_test = train_test_split(
    predictors, target, test_size=0.20, random_state=42) # Seed with 42 for reproducibility

In [ ]:
mod = smf.ols(formula=formula, data = predictor_train.join(target_train))
res = mod.fit()
res.summary()

Woah! An $r^2$ value of 0.986! This model clearly fits the training data very well, though the same may not be the case for the test data. There are other problems, too. The fact that predictors are correlated with each other is causing problems that show up as non-sensical coefficient estimates. There is a negative correlation for bathrooms (!), for example. I'll perform RFE with cross validation in order to obtain a better model, though I imagine that I'll likely end up with the same problems since RFE only looks to optimize $r^2$, RMSE, or other similar indicators. Maybe this is an opportunity to build my own forward selector?


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

In [ ]:
estimator = SVR(kernel = 'linear')
selector = RFECV(estimator = estimator, cv = 3, scoring = 'neg_mean_squared_error')

In [ ]:
selector = selector.fit(predictor_train, target_train)

In [ ]:
RFE_predictors = predictor_train.columns[selector.get_support()]

In [ ]:
formula = 'Price ~ ' + ' + '.join(RFE_predictors) + ' - 1'
print(formula)

In [ ]:
mod = smf.ols(formula=formula, data = predictor_train.join(target_train))
res = mod.fit()
res.summary()

This model is not siginficantly better than the previous with regards to the problems identified.  There are still problems with correlation between predictors, and there are some predictors whose coefficients are not statistically significant. It's a shame that SKLearn's RFE can't catch this. I will eliminate the predictors that are insignificant and one of the offending multicollinear predictors (Bathrooms), and then rebuild the model.

In [ ]:
RFE_predictors = list(RFE_predictors)
RFE_predictors.remove('Bathrooms')

In [ ]:
formula = 'Price ~ ' + ' + '.join(RFE_predictors) + ' - 1'
print(formula)

In [ ]:
mod = smf.ols(formula=formula, data = predictor_train.join(target_train))
res = mod.fit()
res.summary()

In [ ]:
y_hat = res.predict(predictor_test)
residuals = target_test - y_hat
RMSE_test = np.sqrt((residuals**2).sum()/len(residuals))
RMSE_train = np.sqrt(res.mse_resid)
print('RMSE of test data:\t' + str(RMSE_test))
print('RMSE of train data:\t' + str(RMSE_train))

The MSE of the train and test dataset are nearly identical, which tells me that the test data is well approximated by a model derived from the training data. I'll look at the residuals below to check for heteroscedasticity and non-random patterns.

In [ ]:
@interact
def residuals_plot(x = RFE_predictors):
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    sns.scatterplot(x = predictor_test[x], y = residuals, ax = ax)
    ax.set_title('Residual plot for ' + x)
    ax.set_ylabel('Residual')
    corr = np.corrcoef(predictor_test[x],residuals)
    corr_text = 'corr: {:1.3f}'.format(corr[1,0])
    plt.text(.875, 
             0.95, 
             corr_text, 
             horizontalalignment='left',
             verticalalignment='top',
             fontsize = 16,
             transform=ax.transAxes)
    plt.show()

The residuals look pretty good. There is some pattern for a few of the predictors, but the correlation value does not exceed 0.254 in magnitude. I'll look at the residual distribution as well. I expect the distribution to be roughly normal.

In [ ]:
residuals.hist(bins = 20, figsize = (12,8))
plt.title('Histogram of Residuals')
plt.xlabel('Residual of transformed and scaled price')
plt.show()

Great, the residuals are in fact normally distributed. I'm curious about how the predictions look as compared to the actual data. I'll make a plot of that for the training set and the test set, and then I'll move on to making a function that predicts the price in dollars.

In [ ]:
@interact
def residual_data_plot(Show = ['Train', 'Test']):
    df_dict = {'Train':[predictor_train, target_train],'Test':[predictor_test,target_test]}
    df_list = df_dict[Show]
    join = df_list[0].copy()
    join['Price'] = df_list[1]
    join.sort_values('Price', inplace = True)
    y_hat = res.predict(join.drop(labels = 'Price', axis = 1))
    MSE = np.mean(np.square(join.Price-y_hat))
    RMSE = np.sqrt(MSE)
    
    # Plot ordered y and y_hat
    fig, ax = plt.subplots(1,1,figsize = (12,8))
    plt.plot(np.array(y_hat), 'bo', markersize = 0.5, label = 'Predicted Price')
    plt.plot(np.array(join.Price),'r-', label = 'Actual Price')
    plt.legend(prop = {'size':12})
    plt.xlabel('Houses Ordered by Price', fontdict = {'size':16})
    plt.ylabel('Transformed and Scaled Price', fontdict = {'size':16})
    title = 'Difference between actual and predicted price in the ' + Show.lower() + ' dataset'
    plt.title(title, fontdict = {'size':18})
    RMSE_text = 'RMSE: {:1.4f}'.format(RMSE)
    plt.text(.025, 
             0.89, 
             RMSE_text, 
             horizontalalignment='left',
             verticalalignment='top',
             fontsize = 14,
             transform=ax.transAxes)
    plt.show()
    

In [ ]:
def predict_price(df, alpha = 0.05, kind = 'mean'):
    # Transform the data
    df_transformed = df.copy()
    transform = {'Bedrooms':np.log10, 
                 'Bathrooms':np.sqrt, 
                 'Grade':np.square, 
                 'Sqft_Above':np.log10, 
                 'Sqft_Basement':np.sqrt,
                 'Zipcode_Means':np.log10}

    for column in list(df.columns):
        if column in transform.keys():
            df_transformed[column] = df_transformed[column].apply(transform[column])
    
    # Normalize the data
    kct = kc_transformed.copy()
    scale_factors = {'Bedrooms':{'min':kct.Bedrooms.min(), 'max':kct.Bedrooms.max()},
                     'Bathrooms':{'min':kct.Bathrooms.min(), 'max':kct.Bathrooms.max()},
                     'Grade':{'min':kct.Grade.min(), 'max':kct.Grade.max()},
                     'Sqft_Above':{'min':kct.Sqft_Above.min(), 'max':kct.Sqft_Above.max()}, 
                     'Sqft_Basement':{'min':kct.Sqft_Basement.min(), 'max':kct.Sqft_Basement.max()},
                     'Zipcode_Means':{'min':kct.Zipcode_Means.min(), 'max':kct.Zipcode_Means.max()}}
    
    df_transformed_scaled = df_transformed.copy()
    to_scale = scale_factors.keys()
    for column in list(to_scale):
        data = df_transformed_scaled[column]
        df_transformed_scaled[column] = (data - scale_factors[column]['min'])/(scale_factors[column]['max']-scale_factors[column]['min'])
    
    # Obtain the price interval
    if kind.lower() == 'mean':
        columns = ['mean_ci_lower', 'mean_ci_upper']
    elif kind.lower() == 'obs':
        columns = ['obs_ci_lower', 'obs_ci_upper']
    else:
        raise Exception('Kind can only be "mean" or "obs".')
        
    predictions = res.get_prediction(df_transformed_scaled).summary_frame(alpha = alpha)
    predictions = predictions.loc[:,columns]
    predictions.columns = ['Lower Price Boundary','Upper Price Boundary']
    
    # Denormalize the price interval (also hardcoded based on the dataset)
    predictions = predictions*(kct.Price.max()-kct.Price.min())+kct.Price.min()
    
    # Transform the price interval and return
    predictions = 10**predictions
    predictions['id'] = df.index
    predictions.set_index('id')
    
    return predictions.join(df, on = 'id').set_index('id')

In [ ]:
predict_price(kc.head(10),alpha = 0.05, kind = 'obs')

The function `predict_price` provides a range of prices based on the input parameters. An interval for the mean price of a house with a set of attributes can be obtained by passing `kind = 'mean'`, whereas a predictive interval for a single observation can be obtained by passing `kind = 'obs'`.

<a id="5"></a>
## 5. Interpreting the Results

here are some results